# In-sensor reservoir computing for language learning via two-dimensional memristors

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

In [2]:
# 从Excel文件中加载数据到DataFrame
data = pd.read_excel("./data.xlsx")

# 将电导值从西门子 (S) 转换为纳西门子 (nS)
data *= 1E9

## Make train dataset
使用不同标准差的高斯分布对神经元输出电流进行建模

In [3]:
# 设置标准差为 0.1
Standard_deviation = 0.1

# 创建一个空的 DataFrame 用于存储训练数据
train_data_01 = pd.DataFrame()

# data.xlsx 文件应包含至少 25 行数据，总共5列。行代表电流值，列代表脉冲
# 把每5个脉冲输出的电流取最大值和最小值的均值形成正态分布，代表每个字的一行。
# 使用正态分布的随机数据模拟实验或观测数据的随机性

# 循环生成五组数据，每一个cell代表一个神经元
# letter代表由5行电流值对应的一个字母
for i in range(0, 5):
    # 创建 'LETTER' 列，生成服从正态分布的随机数据，均值为 i，标准差为 0
    data0 = pd.DataFrame({'LETTER': np.random.normal(i, 0, size=100)})
    
    # 生成 'CELL1' 列，数据的均值是 data 中第 5*i 行的最大值和最小值的均值，标准差为 0.1
    data1 = pd.DataFrame({'CELL1': np.random.normal(np.mean([np.max(data.iloc[5*i+0]), np.min(data.iloc[5*i+0])]), 0.1, size=100)})
    
    # 生成 'CELL2' 列，数据的均值是 data 中第 5*i+1 行的最大值和最小值的均值，标准差为 0.1
    data2 = pd.DataFrame({'CELL2': np.random.normal(np.mean([np.max(data.iloc[5*i+1]), np.min(data.iloc[5*i+1])]), 0.1, size=100)})
    
    # 生成 'CELL3' 列，数据的均值是 data 中第 5*i+2 行的最大值和最小值的均值，标准差为 0.1
    data3 = pd.DataFrame({'CELL3': np.random.normal(np.mean([np.max(data.iloc[5*i+2]), np.min(data.iloc[5*i+2])]), 0.1, size=100)})
    
    # 生成 'CELL4' 列，数据的均值是 data 中第 5*i+3 行的最大值和最小值的均值，标准差为 0.1
    data4 = pd.DataFrame({'CELL4': np.random.normal(np.mean([np.max(data.iloc[5*i+3]), np.min(data.iloc[5*i+3])]), 0.1, size=100)})
    
    # 生成 'CELL5' 列，数据的均值是 data 中第 5*i+4 行的最大值和最小值的均值，标准差为 0.1
    data5 = pd.DataFrame({'CELL5': np.random.normal(np.mean([np.max(data.iloc[5*i+4]), np.min(data.iloc[5*i+4])]), 0.1, size=100)})
    
    # 将 data0, data1, data2, data3, data4, data5 沿列方向合并，然后将其添加到训练数据集中
    train_data_01 = pd.concat([train_data_01, pd.concat([data0, data1, data2, data3, data4, data5], axis=1)])

# 显示生成的训练数据
display(train_data_01)

# 将训练数据保存为 CSV 文件，编码方式为 CP949，且不保存索引
train_data_01.to_csv("train_data_01.csv", encoding="CP949", index=False)

## Graph of data distribution (Train dataset)

In [4]:
# 创建一个空的 DataFrame 用于存储所有数据
total_data = pd.DataFrame()

# 循环处理五组数据
for i in range(0, 5):
    # 选取 train_data_01 中的第 i 组数据（100 行数据）
    total = train_data_01[100*i:100*(i+1)]
    
    # 将选定的数据从宽格式转换为长格式，只保留 'CELL1', 'CELL2', 'CELL3', 'CELL4', 'CELL5' 列
    total = pd.melt(total, value_vars=['CELL1', 'CELL2', 'CELL3', 'CELL4', 'CELL5'])
    
    # 将每次处理的数据连接到 total_data 中，沿列方向合并
    total_data = pd.concat([total_data, total], axis=1)

# 显示合并后的总数据
display(total_data)

In [5]:
import matplotlib.pyplot as plt 
import seaborn as sns 

# 循环绘制五组数据的分布图
for i in range(0, 5):
    
    # 定义细胞列名称
    Cells = ['CELL1', 'CELL2', 'CELL3', 'CELL4', 'CELL5']
    
    # 设置画布大小
    plt.figure(figsize=(20, 6))    
    
    # 设置 Seaborn 图形样式
    sns.set(font_scale=1.5)  # 设置字体缩放比例
    sns.set_style("whitegrid")  # 设置网格样式
    sns.set_style("white")  # 设置白色背景样式（覆盖之前的样式）
    
    # 绘制每个细胞数据的概率密度曲线
    for Num in Cells:
        # 选取对应细胞的子集数据，利用 'variable' 列筛选数据
        subset = total_data.iloc[:, (2*i):(2*(i+1))][total_data.iloc[:, (2*i):(2*(i+1))]['variable'] == Num]

        # 使用 Seaborn 绘制概率密度图（核密度估计）
        sns.distplot(subset['value'], hist=False, kde=True,
                     kde_kws={'linewidth': 1},
                     label=Num)   
    
    # 设置图表标题，显示当前字母的编号
    plt.title('Letter' + str(i+1), fontsize=25)
    
    # 设置图例
    plt.legend(fontsize=15, loc='upper right')
    
    # 设置 x 轴标签
    plt.xlabel('Conductance(nS)', fontsize=18)

## Make test dataset

In [6]:
# 设置标准差为 0.1
Standard_deviation = 0.1

# 创建一个空的 DataFrame 用于存储测试数据
test_data_01 = pd.DataFrame()

# 循环生成五组数据
for i in range(0, 5):
    # 创建 'LETTER' 列，生成服从正态分布的随机数据，均值为 i，标准差为 0
    data0 = pd.DataFrame({'LETTER': np.random.normal(i, 0, size=100)})
    
    # 生成 'CELL1' 列，数据的均值是 data 中第 5*i 行的最大值和最小值的均值，标准差为 0.1
    data1 = pd.DataFrame({'CELL1': np.random.normal(np.mean([np.max(data.iloc[5*i+0]), np.min(data.iloc[5*i+0])]), 0.1, size=100)})
    
    # 生成 'CELL2' 列，数据的均值是 data 中第 5*i+1 行的最大值和最小值的均值，标准差为 0.1
    data2 = pd.DataFrame({'CELL2': np.random.normal(np.mean([np.max(data.iloc[5*i+1]), np.min(data.iloc[5*i+1])]), 0.1, size=100)})
    
    # 生成 'CELL3' 列，数据的均值是 data 中第 5*i+2 行的最大值和最小值的均值，标准差为 0.1
    data3 = pd.DataFrame({'CELL3': np.random.normal(np.mean([np.max(data.iloc[5*i+2]), np.min(data.iloc[5*i+2])]), 0.1, size=100)})
    
    # 生成 'CELL4' 列，数据的均值是 data 中第 5*i+3 行的最大值和最小值的均值，标准差为 0.1
    data4 = pd.DataFrame({'CELL4': np.random.normal(np.mean([np.max(data.iloc[5*i+3]), np.min(data.iloc[5*i+3])]), 0.1, size=100)})
    
    # 生成 'CELL5' 列，数据的均值是 data 中第 5*i+4 行的最大值和最小值的均值，标准差为 0.1
    data5 = pd.DataFrame({'CELL5': np.random.normal(np.mean([np.max(data.iloc[5*i+4]), np.min(data.iloc[5*i+4])]), 0.1, size=100)})
    
    # 将 data0, data1, data2, data3, data4, data5 沿列方向合并，然后将其添加到测试数据集中
    test_data_01 = pd.concat([test_data_01, pd.concat([data0, data1, data2, data3, data4, data5], axis=1)])

# 显示生成的测试数据
display(test_data_01)

# 将测试数据保存为 CSV 文件，编码方式为 CP949，且不保存索引
test_data_01.to_csv("test_data_01.csv", encoding="CP949", index=False)

## Graph of data distribution (Test dataset)

In [7]:
# 创建一个空的 DataFrame 用于存储所有数据
total_data2 = pd.DataFrame()

# 循环处理十五组数据
for i in range(0, 15):
    # 选取 test_data_01 中的第 i 组数据（100 行数据）
    total2 = test_data_01[100*i:100*(i+1)]
    
    # 将选定的数据从宽格式转换为长格式，只保留 'CELL1', 'CELL2', 'CELL3', 'CELL4', 'CELL5' 列
    total2 = pd.melt(total2, value_vars=['CELL1', 'CELL2', 'CELL3', 'CELL4', 'CELL5'])
    
    # 将每次处理的数据连接到 total_data2 中，沿列方向合并
    total_data2 = pd.concat([total_data2, total2], axis=1)

# 显示合并后的总数据
display(total_data2)


In [8]:
# 循环绘制五组数据的分布图
for i in range(0, 5):
    
    # 定义细胞列名称
    Cells = ['CELL1', 'CELL2', 'CELL3', 'CELL4', 'CELL5']
    
    # 设置画布大小
    plt.figure(figsize=(20, 6))    
    
    # 设置 Seaborn 图形样式
    sns.set(font_scale=1.5)  # 设置字体缩放比例
    sns.set_style("whitegrid")  # 设置网格样式
    sns.set_style("white")  # 设置白色背景样式（覆盖之前的样式）
    
    # 绘制每个细胞数据的概率密度曲线，横轴和纵轴分别表示数据值的范围和概率密度
    for Num in Cells:
        # 选取对应细胞的子集数据，利用 'variable' 列筛选数据
        subset = total_data2.iloc[:, (2*i):(2*(i+1))][total_data2.iloc[:, (2*i):(2*(i+1))]['variable'] == Num]

        # 使用 Seaborn 绘制概率密度图（核密度估计）
        sns.distplot(subset['value'], hist=False, kde=True,
                     kde_kws={'linewidth': 1},
                     label=Num)   
    
    # 设置图表标题，显示当前字母的编号
    plt.title('Letter' + str(i+1), fontsize=25)
    
    # 设置图例
    plt.legend(fontsize=15, loc='upper right')
    
    # 设置 x 轴标签
    plt.xlabel('Conductance(nS)', fontsize=18)